<a href="https://colab.research.google.com/github/mori8/NLP-laboratory/blob/main/word2vec_skip_gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. One Hot Vector 형태의 입력값을 받는다.
- [say] -> [you] 예측
- [say] -> [goodbye] 예측
  - 주변 단어: 주변에 있는 단어(you, goodbye)
  - 중심 단어: 중간에 있는 단어(say)
  - 윈도우 크기: 주변 몇 칸까지 볼 지에 대한 크기(여기서는 1)

2. one-hot vector 형태의 입력값을 W_in과 곱한다.

In [31]:
# 문장: You say goodbye and I say hello.
# 단어: you, say, goodbye, and, I, hello, .

In [32]:
import numpy as np

input_vector = np.array([0, 1, 0, 0, 0, 0, 0]) # say

output1 = np.array([[1, 0, 0, 0, 0, 0, 0]]) # you
output2 = np.array([[0, 0, 1, 0, 0, 0, 0]]) # goodbye

In [33]:
# (입력 x 차원의 크기) - 차원의 크기는 사용자가 선정
# 초기의 weight는 랜덤하게 결정됨
W_in = np.random.randn(7, 3)

In [34]:
h = np.matmul(input_vector, W_in) # h = 은닉층의 값

In [35]:
print(h)

[-2.56575175 -0.39806727 -1.45554442]


3. hidden state의 값을 W_out과 곱해 score를 계산한다.
4. score에 softmax를 취해서 각 단어가 나올 **확률**을 계산한다.

In [36]:
W_out = np.random.randn(3, 7)
score = np.matmul(h, W_out)
print(np.round(score, 4))

[-4.7568  3.6403 -3.1095  0.0146 -4.425  -0.1039  2.1312]


In [37]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [38]:
pred = softmax(score)
print(np.round(pred, 4))

[2.000e-04 7.855e-01 9.000e-04 2.090e-02 2.000e-04 1.860e-02 1.737e-01]


5. 정답과 Cross Entropy Loss를 계산한다.
- 예측: [[**0.1127** 0.0112 **0.5074** 0.0046 0.0084 0.01 0.3457]]
- 정답: [**1**, 0, **1**, 0, 0, 0, 0]
  - 한번에 업데이트 진행

6. 5에서 계산한 Loss를 가지고 Backpropagation 과정을 통해서 Weight를 업데이트
- 두 개의 결과(윈도우 크기가 1이니 양쪽에서 한 개씩 총 2개)에 대한 오차를 더함
- W_out에 대해 역전파값을 계산
- W_in에 대해서 Update 진행

In [39]:
ds1 = np.round(pred - output1, 4)
ds2 = np.round(pred - output2, 4)
ds = ds1 + ds2
print(ds)

[[-9.996e-01  1.571e+00 -9.982e-01  4.180e-02  4.000e-04  3.720e-02
   3.474e-01]]


In [40]:
dw_out = np.round(np.outer(h, ds), 4)
print(dw_out)

[[ 2.5647e+00 -4.0308e+00  2.5611e+00 -1.0720e-01 -1.0000e-03 -9.5400e-02
  -8.9130e-01]
 [ 3.9790e-01 -6.2540e-01  3.9740e-01 -1.6600e-02 -2.0000e-04 -1.4800e-02
  -1.3830e-01]
 [ 1.4550e+00 -2.2867e+00  1.4529e+00 -6.0800e-02 -6.0000e-04 -5.4100e-02
  -5.0570e-01]]


In [41]:
da = np.dot(ds, W_out.T)
print(np.round(da, 4))

[[-3.6521 -3.3967 -2.4668]]


In [42]:
dw_in = np.outer(np.array([0, 1, 0, 0, 0, 0, 0]), da)
print(np.round(dw_in, 4))

[[-0.     -0.     -0.    ]
 [-3.6521 -3.3967 -2.4668]
 [-0.     -0.     -0.    ]
 [-0.     -0.     -0.    ]
 [-0.     -0.     -0.    ]
 [-0.     -0.     -0.    ]
 [-0.     -0.     -0.    ]]


In [43]:
# W_in에 대해서 Update 진행
learning_rate = 1
W_in_new = W_in - learning_rate * dw_in
print(np.round(W_in_new, 4))

[[ 1.0000e-04  8.0730e-01 -2.3840e+00]
 [ 1.0864e+00  2.9987e+00  1.0113e+00]
 [-5.3300e-02 -1.8743e+00 -5.4310e-01]
 [ 1.1630e-01  4.9160e-01  1.8909e+00]
 [ 9.5220e-01 -9.3220e-01  1.0922e+00]
 [-6.7350e-01  1.0654e+00 -1.0437e+00]
 [ 1.0761e+00  1.8980e-01 -6.5490e-01]]


In [44]:
print(np.round(W_in, 4))

[[ 1.0000e-04  8.0730e-01 -2.3840e+00]
 [-2.5658e+00 -3.9810e-01 -1.4555e+00]
 [-5.3300e-02 -1.8743e+00 -5.4310e-01]
 [ 1.1630e-01  4.9160e-01  1.8909e+00]
 [ 9.5220e-01 -9.3220e-01  1.0922e+00]
 [-6.7350e-01  1.0654e+00 -1.0437e+00]
 [ 1.0761e+00  1.8980e-01 -6.5490e-01]]


7. 위 과정을 다른 문맥에 대해서도 반복 수행

### gensim 패키지의 Word2Vec을 이용

In [45]:
from gensim.models import Word2Vec

docs = ['you say goodbye and I say hello.']

sentences = [list(sentence.split(' ')) for sentence in docs]
model = Word2Vec(size=3, window=1, min_count=1, sg=1)
model.build_vocab(sentences)
model.wv.most_similar("say")

[('and', 0.9665473699569702),
 ('you', 0.5921512246131897),
 ('hello.', 0.4940755069255829),
 ('goodbye', 0.4343804717063904),
 ('I', -0.7611031532287598)]

In [ ]:
# https://www.youtube.com/watch?v=3jfHP0Rq1Gg